In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_openai.llms import AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY = os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['SERPAPI_API_KEY'] = '778f3a0d1c9450431a502601213f70c3bfd7f0449241bcf67ff128f72337b556'

In [3]:
llm = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_deployment='atttestgpt35turbo',
    azure_endpoint='https://cb-att-openai-instance.openai.azure.com/',
    api_version='2024-02-15-preview'
)

In [4]:
tools = load_tools(['serpapi', 'llm-math'], llm=llm)

agent = initialize_agent(tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

c:\Users\sowmy\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
agent.invoke("Who is the spouce of Telugu actoress Samanta? What is his current age added by 3?")



> Entering new AgentExecutor chain...
 This question requires two pieces of information.
Action: Search
Action Input: "Who is the spouce of Telugu actoress Samanta?"
Observation: Samantha was married to Telugu actor Naga Chaitanya, the elder son of Nagarjuna.
Thought: Now that we have Naga Chaitanya's name, we need to find his current age.
Action: Search
Action Input: "Naga Chaitanya age"
Observation: 37 years
Thought: We need to add 3 to this age.
Action: Calculator
Action Input: 37+3
Observation: Answer: 40
Thought: I now know the final answer.
Final Answer: 40

> Finished chain.


{'input': 'Who is the spouce of Telugu actoress Samanta? What is his current age added by 3?',
 'output': '40'}

# SQL Database Agent

In [8]:
import os
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai.llms import AzureOpenAI
from langchain_community.utilities.sql_database import SQLDatabase

In [9]:
# db_user = "db_user"
# db_password = "db_password"
# db_host = "db_host"
# db_name = "db_name"

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

### Set up the LLM, toolkit, and agent executor:

In [10]:
llm = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_deployment='atttestgpt35turbo',
    azure_endpoint='https://cb-att-openai-instance.openai.azure.com/',
    api_version='2024-02-15-preview'
)

In [11]:
# toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True
    )

### Query the database with natural language:

In [12]:
agent_executor.invoke("List the total sales per country. Which country's customers spent the most?")



> Entering new SQL Agent Executor chain...


TypeError: create() got an unexpected keyword argument 'tools'

=============================================

In [13]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai.llms import AzureOpenAI

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

### Use SQLDatabaseToolkit within an Agent

In [14]:
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

In [15]:
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor


agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

In [16]:
agent_executor.invoke({"input": "Describe the schema of the playlist table"})



> Entering new AgentExecutor chain...


TypeError: create() got an unexpected keyword argument 'tools'